In [1]:
import numpy as np
import numpy as np
import scipy.sparse
from scipy.sparse import lil_matrix

In [2]:
import DataFileIO
FileIO = DataFileIO.DataIO()

In [3]:
canonical_users = FileIO.GetUserList()
song_to_index = FileIO.GetSongToIndex()
user_to_index = FileIO.GetUserIndex(canonical_users)
print(len(user_to_index))

Get User List ...
Get Song Index ...
Get User ID ...
110000


In [4]:
song_to_count = FileIO.GetSongToCount(song_to_index)
column_to_song_index = dict()
song_to_column_index = dict()
song_ptr = 0
for song in iter(song_to_index):
    if int(song_to_index[song]) in song_to_count:
        temp_song = int(song_to_index[song])
        column_to_song_index[song_ptr] = temp_song
        song_to_column_index[temp_song] = song_ptr
        song_ptr += 1

Get Each Song Count ...


In [5]:
user_count = len(user_to_index)
song_count = len(song_to_column_index)
user_to_song =lil_matrix((user_count, song_count))
print("Get Song item and its listener.\nTotal User = %d, Total Song = %d" % (user_count, song_count))
f = open('data/kaggle_visible_evaluation_triplets.txt', 'r')
for line in f:
    user, song, times = line.strip().split('\t')
    user_to_song[int(user_to_index[user]), song_to_column_index[int(song_to_index[song])]] = float(times) - 1.0
print("Finish Getting All Song and its record")

Get Song item and its listener.
Total User = 110000, Total Song = 163206
Finish Getting All Song and its record


In [6]:
# initialize x, y and lambda
k = 30
reg_lambda = 0.1
X = np.mat(np.random.random_sample((k, user_count)))
Y = np.mat(np.random.random_sample((k, song_count)))

In [7]:
Iteration_time = 100
Iter = 0

In [8]:
while Iter < Iteration_time:
    Y_sum = np.mat(np.eye(k) * reg_lambda)
    Y_sum += Y * Y.T
    Y_sum = Y_sum.I * Y
    X = Y_sum * user_to_song.T
    #for u in range(user_count):
    #    X[:, u] = Y_sum * (user_to_song[u,:].T)
    #    print(u)
    
    X_sum = np.mat(np.eye(k) * reg_lambda)
    X_sum += X * X.T
    X_sum = X_sum.I * X
    #for i in range(song_count):
    #    Y[:, i] = X_sum * (user_to_song[:, i])
    Y = X_sum * user_to_song
        
    Iter += 1

In [9]:
f= open('submission.txt', 'w')
for user in range(10000):
    result = X[:,user].T * Y
    rated_item = user_to_song[user].rows[0]
    indices = np.argsort(result, axis = 1)
    recommend = []
    for i in range(500):
        song = indices[0, len(column_to_song_index) - 1 - i]
        #print(type(song))
        if song not in rated_item:
            recommend.append(str(column_to_song_index[song]))
    f.write(' '.join(recommend) + '\n')
    if user % 1000 == 0:
        print(user)
f.close()

0
1000
2000
3000
4000
5000
6000
7000
8000
9000


In [10]:
print("Reading Testing Data...")
f = open('data/kaggle_hidden_evaluation_triplets.txt', 'r')
result_user_to_song = dict()
for line in f:
    user, song, _ = line.strip().split('\t')
    if user in result_user_to_song:
        result_user_to_song[user].add(song_to_index[song])
    else:
        result_user_to_song[user] = set([song_to_index[song]])
f.close()

Reading Testing Data...


In [11]:
f = open('submission.txt', 'r')
print("Begin Testing ...")
predicted_user_to_song = dict()
ptr = 0
for line in f:
    predicted_user_to_song[canonical_users[ptr]] = list()
    predicted_user_to_song[canonical_users[ptr]] = line.strip().split(' ')
    ptr += 1
f.close()

Begin Testing ...


In [12]:
score = 0
canonical_sub_users = []
for i in range(10000):
    canonical_sub_users.append(canonical_users[i])
for user in canonical_sub_users:
    pre_song = predicted_user_to_song[user]
    res_song = result_user_to_song[user]
    correct_song_number = 0
    for i in range(len(pre_song)):
        if pre_song[i] in res_song:
            correct_song_number += 1
            rel = 1
        else:
            rel = 0
        min_num = min(len(pre_song), len(res_song))
        score += rel * correct_song_number / (i + 1) / min_num
    print("%d / %d"%(correct_song_number, len(res_song) ))
print(score)
print("Accurate rate = %.8f"% (score / 10000))

0 / 7
0 / 16
0 / 7
0 / 11
0 / 15
1 / 10
2 / 10
5 / 8
0 / 9
2 / 11
0 / 11
1 / 9
1 / 6
0 / 7
0 / 8
2 / 36
5 / 25
2 / 18
2 / 19
0 / 25
9 / 24
1 / 5
2 / 14
1 / 20
6 / 21
2 / 9
4 / 16
6 / 6
1 / 5
2 / 5
0 / 27
0 / 6
19 / 28
0 / 6
6 / 18
0 / 8
5 / 8
0 / 11
0 / 12
2 / 7
5 / 12
2 / 7
1 / 18
3 / 21
2 / 14
1 / 5
2 / 34
1 / 20
1 / 6
0 / 6
0 / 6
0 / 6
0 / 22
1 / 7
5 / 22
2 / 8
2 / 21
5 / 15
3 / 5
0 / 7
1 / 9
2 / 11
1 / 12
0 / 6
5 / 25
1 / 18
5 / 36
0 / 13
1 / 8
10 / 40
1 / 10
2 / 26
0 / 22
0 / 6
1 / 14
4 / 17
3 / 8
0 / 11
1 / 10
0 / 7
4 / 29
0 / 11
0 / 14
1 / 9
0 / 13
2 / 5
1 / 20
0 / 10
1 / 12
1 / 9
0 / 11
1 / 5
3 / 8
0 / 15
1 / 9
1 / 11
0 / 6
6 / 26
1 / 16
0 / 7
5 / 17
2 / 6
2 / 22
11 / 15
3 / 6
2 / 18
2 / 10
2 / 19
1 / 22
0 / 21
5 / 44
4 / 18
2 / 16
0 / 35
0 / 9
1 / 14
3 / 8
5 / 8
2 / 7
7 / 24
0 / 15
0 / 6
1 / 7
0 / 11
7 / 15
1 / 8
5 / 18
0 / 19
0 / 7
0 / 11
3 / 20
6 / 37
2 / 11
0 / 11
5 / 27
0 / 6
0 / 7
0 / 14
0 / 11
2 / 12
5 / 7
4 / 24
0 / 8
4 / 7
0 / 8
1 / 14
0 / 26
3 / 8
3 / 14
0 / 5
1 / 5
1

9 / 40
1 / 14
2 / 26
8 / 37
2 / 10
5 / 10
1 / 15
0 / 22
1 / 6
0 / 9
2 / 32
0 / 8
1 / 10
2 / 44
0 / 9
0 / 6
1 / 9
1 / 27
10 / 17
0 / 8
0 / 9
6 / 27
0 / 7
1 / 7
0 / 10
0 / 7
3 / 37
1 / 24
1 / 6
9 / 18
2 / 6
3 / 32
1 / 11
3 / 13
0 / 32
2 / 13
4 / 5
3 / 5
0 / 6
5 / 17
2 / 24
4 / 22
2 / 25
3 / 27
0 / 22
1 / 5
4 / 6
4 / 17
0 / 22
2 / 7
1 / 10
0 / 7
14 / 16
2 / 16
0 / 18
1 / 26
0 / 11
0 / 22
0 / 7
4 / 12
1 / 11
25 / 33
2 / 12
16 / 18
4 / 7
0 / 13
2 / 15
0 / 6
0 / 11
4 / 7
10 / 15
0 / 7
2 / 32
0 / 7
7 / 12
2 / 6
0 / 5
7 / 38
0 / 19
1 / 9
0 / 11
1 / 12
4 / 5
0 / 6
0 / 12
0 / 7
3 / 13
0 / 7
1 / 7
0 / 6
9 / 13
2 / 15
0 / 8
0 / 8
4 / 9
13 / 23
7 / 19
2 / 17
1 / 6
1 / 14
0 / 12
0 / 18
1 / 10
1 / 8
0 / 9
0 / 9
1 / 9
0 / 9
2 / 14
0 / 8
8 / 38
1 / 7
1 / 7
0 / 11
1 / 28
2 / 30
5 / 15
0 / 14
0 / 11
1 / 8
4 / 19
0 / 11
3 / 9
2 / 14
0 / 10
18 / 19
1 / 7
0 / 8
8 / 10
9 / 9
1 / 24
2 / 6
6 / 11
2 / 16
1 / 5
0 / 11
0 / 19
1 / 20
0 / 7
0 / 17
0 / 8
2 / 29
0 / 8
2 / 7
5 / 27
9 / 13
4 / 6
0 / 8
2 / 6
2 / 8
3 / 5

0 / 8
0 / 16
2 / 6
0 / 9
8 / 24
4 / 5
1 / 7
0 / 6
0 / 5
2 / 8
4 / 34
1 / 9
6 / 10
1 / 20
2 / 17
4 / 18
4 / 16
2 / 17
1 / 14
1 / 23
1 / 6
1 / 28
3 / 14
2 / 19
16 / 17
0 / 14
1 / 23
0 / 9
3 / 13
3 / 8
0 / 7
0 / 7
0 / 40
3 / 16
0 / 5
0 / 6
15 / 19
2 / 7
2 / 14
3 / 11
0 / 7
0 / 7
1 / 7
2 / 17
0 / 5
2 / 9
1 / 9
0 / 6
1 / 5
0 / 5
0 / 14
1 / 8
0 / 16
0 / 18
4 / 6
0 / 7
2 / 28
1 / 40
0 / 24
13 / 17
0 / 16
2 / 7
1 / 7
1 / 6
2 / 8
3 / 8
0 / 10
0 / 26
3 / 10
10 / 18
0 / 11
6 / 30
2 / 8
5 / 30
0 / 11
0 / 7
3 / 22
0 / 6
3 / 15
0 / 29
1 / 5
0 / 25
1 / 6
0 / 13
8 / 20
0 / 7
2 / 11
1 / 8
0 / 8
4 / 6
0 / 8
3 / 29
0 / 6
0 / 5
1 / 11
0 / 9
0 / 10
0 / 11
0 / 32
0 / 12
4 / 8
0 / 6
0 / 5
0 / 9
0 / 11
1 / 22
0 / 10
7 / 14
1 / 13
2 / 16
0 / 24
2 / 11
1 / 18
1 / 20
0 / 6
2 / 8
4 / 12
0 / 6
1 / 6
0 / 22
1 / 8
4 / 20
0 / 5
1 / 18
4 / 12
0 / 20
0 / 14
3 / 12
0 / 8
1 / 16
3 / 5
0 / 6
0 / 5
1 / 8
0 / 9
5 / 10
1 / 28
0 / 30
0 / 8
0 / 17
2 / 21
0 / 11
2 / 9
1 / 13
3 / 12
6 / 29
3 / 7
5 / 21
0 / 7
0 / 11
3 / 20
3 / 11

5 / 9
0 / 8
4 / 10
0 / 6
0 / 9
0 / 36
0 / 9
2 / 9
0 / 10
5 / 34
2 / 16
0 / 7
0 / 6
0 / 8
2 / 27
1 / 9
2 / 9
2 / 7
0 / 17
0 / 15
0 / 6
2 / 7
0 / 7
7 / 20
1 / 11
0 / 17
2 / 30
0 / 10
0 / 8
1 / 6
1 / 9
1 / 13
6 / 18
2 / 24
0 / 6
0 / 11
1 / 9
0 / 11
0 / 5
0 / 6
0 / 9
1 / 10
1 / 12
5 / 21
2 / 6
12 / 25
0 / 8
4 / 16
3 / 41
7 / 27
1 / 9
0 / 10
3 / 20
5 / 12
0 / 18
6 / 12
6 / 13
5 / 22
0 / 6
0 / 13
6 / 35
4 / 5
2 / 16
0 / 6
1 / 5
0 / 5
4 / 11
0 / 6
4 / 12
9 / 16
0 / 9
1 / 10
3 / 27
1 / 7
0 / 7
4 / 29
1 / 8
1 / 7
0 / 8
4 / 14
1 / 9
0 / 8
0 / 6
1 / 15
2 / 6
0 / 10
2 / 7
2 / 17
0 / 8
1 / 26
6 / 18
9 / 25
0 / 5
0 / 8
4 / 21
0 / 7
2 / 10
5 / 16
3 / 13
9 / 25
8 / 10
2 / 12
1 / 13
0 / 14
2 / 22
3 / 20
0 / 7
1 / 16
26 / 37
1 / 7
3 / 8
0 / 11
6 / 28
0 / 5
2 / 24
2 / 9
3 / 17
0 / 15
1 / 11
0 / 16
2 / 12
3 / 12
0 / 7
0 / 19
4 / 10
1 / 21
0 / 15
4 / 12
2 / 20
3 / 8
2 / 6
1 / 16
1 / 16
4 / 21
2 / 8
1 / 16
2 / 22
4 / 6
3 / 9
1 / 25
0 / 10
0 / 6
4 / 16
5 / 8
4 / 13
1 / 23
5 / 14
0 / 7
2 / 17
6 / 12
1 / 6
0 /

3 / 7
0 / 9
2 / 14
8 / 8
0 / 10
15 / 26
0 / 12
5 / 22
0 / 18
0 / 5
3 / 7
1 / 15
2 / 7
0 / 6
0 / 7
0 / 8
2 / 11
0 / 9
0 / 9
2 / 6
0 / 8
6 / 14
0 / 5
2 / 33
0 / 14
3 / 14
5 / 25
2 / 7
0 / 11
4 / 20
0 / 9
6 / 22
0 / 6
0 / 6
0 / 10
1 / 20
0 / 10
2 / 14
0 / 31
2 / 25
1 / 7
2 / 16
2 / 8
0 / 5
0 / 6
1 / 13
0 / 37
1 / 8
2 / 20
1 / 24
2 / 5
0 / 6
3 / 26
0 / 9
1 / 6
4 / 30
3 / 6
1 / 27
3 / 12
1 / 22
2 / 7
0 / 5
1 / 25
0 / 15
0 / 10
0 / 20
0 / 5
0 / 7
0 / 37
0 / 5
3 / 23
1 / 6
4 / 20
15 / 19
2 / 23
2 / 22
10 / 24
4 / 25
4 / 9
4 / 24
4 / 11
7 / 11
1 / 6
0 / 39
4 / 21
0 / 8
2 / 10
0 / 10
3 / 19
4 / 28
2 / 7
0 / 24
0 / 6
1 / 6
1 / 14
8 / 10
2 / 6
0 / 6
1 / 13
5 / 18
0 / 8
0 / 16
4 / 17
0 / 17
0 / 11
0 / 10
3 / 7
1 / 9
0 / 17
1 / 6
3 / 7
3 / 17
1 / 17
5 / 6
0 / 27
11 / 11
0 / 11
5 / 22
0 / 7
0 / 30
0 / 28
0 / 8
5 / 7
0 / 23
0 / 7
0 / 5
0 / 9
9 / 11
1 / 5
16 / 26
1 / 6
4 / 13
2 / 33
9 / 9
0 / 11
1 / 12
0 / 15
1 / 9
1 / 6
3 / 16
0 / 7
0 / 17
1 / 7
0 / 18
1 / 22
8 / 23
0 / 7
4 / 9
2 / 11
0 / 5
0 / 10
2 

1 / 9
8 / 18
0 / 38
1 / 11
1 / 5
4 / 22
2 / 8
1 / 9
1 / 7
0 / 17
1 / 20
2 / 7
0 / 25
1 / 9
1 / 26
3 / 30
0 / 6
8 / 33
0 / 8
2 / 19
0 / 22
0 / 6
0 / 10
6 / 14
2 / 6
0 / 8
4 / 7
0 / 8
0 / 5
2 / 5
1 / 5
6 / 11
3 / 15
0 / 5
0 / 5
1 / 10
1 / 13
0 / 33
12 / 34
0 / 5
0 / 9
5 / 13
9 / 15
12 / 25
4 / 18
1 / 6
1 / 21
0 / 21
2 / 21
0 / 7
2 / 10
3 / 7
5 / 6
2 / 22
0 / 7
9 / 10
9 / 17
4 / 7
1 / 10
6 / 18
0 / 11
15 / 22
0 / 6
1 / 11
1 / 18
1 / 19
0 / 6
0 / 7
0 / 17
4 / 7
1 / 28
1 / 12
2 / 6
3 / 10
2 / 17
0 / 25
0 / 5
0 / 9
0 / 6
4 / 36
0 / 6
0 / 7
1 / 34
0 / 5
1 / 7
0 / 7
0 / 5
0 / 46
0 / 7
1 / 20
12 / 14
4 / 18
3 / 7
0 / 5
1 / 12
2 / 8
0 / 6
0 / 11
5 / 6
0 / 7
3 / 9
0 / 5
6 / 21
11 / 12
0 / 10
1 / 12
0 / 12
5 / 8
1 / 34
1 / 24
4 / 35
3 / 11
0 / 5
0 / 10
3 / 17
3 / 11
5 / 17
1 / 8
1 / 14
0 / 6
4 / 9
0 / 9
3 / 13
4 / 28
6 / 15
1 / 14
0 / 5
0 / 9
2 / 7
1 / 6
0 / 14
1 / 13
0 / 8
5 / 13
2 / 16
3 / 5
3 / 20
1 / 5
7 / 14
1 / 8
1 / 14
0 / 28
0 / 8
2 / 11
0 / 7
1 / 7
0 / 11
0 / 12
1 / 8
10 / 14
1 / 6
7 / 17

In [24]:
from numpy import linalg as la
def cosSim(inA, inB):
    denom = la.norm(inA) * la.norm(inB)
    if denom == 0:
        return 0
    num = float(np.dot(inA.T, inB))
    return (num / denom)

In [33]:
rated_item = user_to_song[3].rows[0]
result = X[:,3].T * Y
for song in rated_item:
    print(result[0, song])

3.19037802124e-05
0.0699100943933
0.977856873708
0.000467842026337
8.86982075104e-05
0.00167857026682
3.52258552724e-05
0.000527532747227
0.000884720438845
2.61123563284e-05
0.00030752023413


In [29]:
user = 1
rated_item = user_to_song[user].rows[0]
rated_value = user_to_song[user].data[0]
print(rated_item)
print(rated_value)
song_score = []
for i in range(song_count):
    if i in rated_item:
        continue
    song_vector = Y[:, i]
    simTotal, rateTotal = 0.0, 0.0
    for song_ptr in range(len(rated_item)):
        rated_vector = Y[:, rated_item[song_ptr]]
        similarity = cosSim(rated_vector, song_vector)
        rateTotal += similarity * rated_value[song_ptr]
    if rateTotal == 0:
        continue
    song_score.append((i+1, rateTotal))
    # print("%d: %4f"%(i+1, rateTotal))

itemScores = sorted(song_score, key=lambda x:x[1], reverse = True)

[1639, 11840, 59732, 67456, 114600, 136847, 195314, 209057, 218287, 236367, 243462, 283891, 312786, 315821, 329556, 381062]
[1.0, 1.0, 1.0, 2.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]


In [35]:
for i in range(100):
    print("%d:%.5f" % (i, itemScores[i]))

TypeError: only length-1 arrays can be converted to Python scalars

In [45]:
result_item = [6507,214720,116525,274868,98012,118874,94689 ,43643 ,320301 ,173222 ,207415 ,152516 ,23658 ,71718 ,268739 ,226931]
for song in recommend:
    if song in result_item:
        print(song)

94689
274868


In [44]:
result = X[:,1].T * Y
indices = np.argsort(result, axis = 1)
recommend = []
for i in range(100):
    song = indices[0, len(song_to_index) - 1 - i]
    if song not in rated_item:
        recommend.append(song + 1)
print(recommend)

[136848, 37292, 247218, 268474, 94689, 154559, 242529, 224988, 177574, 310418, 296731, 116409, 102739, 315822, 314086, 345265, 279154, 277126, 192914, 305991, 60252, 93787, 358834, 283314, 335200, 297177, 308356, 8402, 142382, 105434, 101204, 242895, 170222, 187097, 326792, 183790, 172688, 176855, 167766, 17637, 148488, 82218, 215861, 69290, 154096, 111771, 236713, 251150, 38941, 12985, 1248, 164552, 385493, 240629, 212017, 161020, 337653, 247288, 337640, 208311, 149450, 303230, 148050, 135913, 238481, 206960, 228613, 217471, 137197, 9615, 288122, 318282, 110212, 153860, 322186, 198994, 98410, 121370, 91546, 54088, 94868, 306687, 130322, 53383, 284759, 295286, 274868, 241283, 91539, 57044, 193937, 200676, 266818, 73628, 343061, 119200, 384130, 239491, 351831, 331326]


In [ ]:
# f= open('submission.txt', 'w')
for user in canonical_users:
    songs_to_recommend = []
    user_ID = user_to_index[user]
    rated_item = user_to_song[user_ID].rows
    rated_value = user_to_song[user_ID].data
    item_score = []
    
    for song in songs_ordered:
        if (len(songs_to_recommend) >= 500):
            break
        if not song in user_to_songs[user]:
            songs_to_recommend.append(song)
            indices = map(lambda s:song_to_index[s], songs_to_recommend)
    f.write(' '.join(indices) + '\n')
f.close()

In [302]:
user_count = 11
song_count = 11
user_to_song = lil_matrix((user_count, song_count))
rate_matrix = np.mat([
    [0, 0, 0, 0, 0, 4, 0, 0, 0, 0, 5],
    [0, 0, 0, 3, 0, 4, 0, 0, 0, 0, 3],
    [0, 0, 0, 0, 4, 0, 0, 1, 0, 4, 0],
    [3, 3, 4, 0, 0, 0, 0, 2, 2, 0, 0],
    [5, 4, 5, 0, 0, 0, 0, 5, 5, 0, 0],
    [0, 0, 0, 0, 5, 0, 1, 0, 0, 5, 0],
    [4, 3, 4, 0, 0, 0, 0, 5, 5, 0, 1],
    [0, 0, 0, 4, 0, 4, 0, 0, 0, 0, 4],
    [0, 0, 0, 2, 0, 2, 5, 0, 0, 1, 2],
    [0, 0, 0, 0, 5, 0, 0, 0, 0, 4, 0],
    [1, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0],
])
user_to_song = rate_matrix

In [322]:
print(rate_matrix)

[[0 0 0 0 0 4 0 0 0 0 5]
 [0 0 0 3 0 4 0 0 0 0 3]
 [0 0 0 0 4 0 0 1 0 4 0]
 [3 3 4 0 0 0 0 2 2 0 0]
 [5 4 5 0 0 0 0 5 5 0 0]
 [0 0 0 0 5 0 1 0 0 5 0]
 [4 3 4 0 0 0 0 5 5 0 1]
 [0 0 0 4 0 4 0 0 0 0 4]
 [0 0 0 2 0 2 5 0 0 1 2]
 [0 0 0 0 5 0 0 0 0 4 0]
 [1 0 0 0 0 0 0 1 2 0 0]]
